In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from data import *
import random

In [2]:
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
def discriminator_bad(dim_length,d=32,input_shape=(64,64,1),name='discriminator',output_dim=10):
    dims=[d *(2**i) for i in range(2,2+dim_length)]
    img_inputs = keras.Input(shape=(input_shape))
    x = layers.Conv2D(d, (4, 4), strides=(2, 2), padding="same",kernel_initializer=initializer)(img_inputs)
    x=layers.LeakyReLU(alpha=0.2)(x)

    for dim in dims:
        x = layers.Conv2D(d, (4, 4), strides=(2, 2), padding="same",kernel_initializer=initializer)(x)
        x= layers.BatchNormalization()(x)
        x=layers.LeakyReLU(alpha=0.2)(x)

    binary_x =layers.Conv2D(d, (4,4),strides=(2,2),kernel_initializer=initializer)(x)
    binary_x=layers.Flatten(name='binary_flatten')(binary_x)
    binary_output = layers.Dense(1,name='binary_output')(binary_x)

    multiclass_x=layers.Dense(d,kernel_initializer=initializer)(x)
    mutliclass_x=layers.Conv2D(dims[-1], (4,4),strides=(2,2),kernel_initializer=initializer)(multiclass_x)
    multiclass_x=layers.Flatten(name='multiclass_flatten')(multiclass_x)
    mutlticlass_output=layers.Dense(output_dim,name='multiclass_output')(multiclass_x)
    model = keras.Model(inputs=img_inputs, outputs=[binary_output,mutlticlass_output], name=name)
    return model

def discriminator(input_shape=(64,64,1),name='discriminator',output_dim=10):
    img_inputs = keras.Input(shape=(input_shape))
    d=input_shape[0]
    #x=layers.Flatten()(img_inputs)
    x=layers.Conv2D(16, (4, 4), strides=(2, 2), padding="same")(img_inputs)
    x=layers.LeakyReLU(alpha=.2)(x)
    x= layers.BatchNormalization()(x)
    x=layers.Conv2D(32, (4, 4), strides=(2, 2), padding="same")(x)
    x=layers.LeakyReLU(alpha=.2)(x)
    x= layers.BatchNormalization()(x)
    x=layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same")(x)
    x=layers.LeakyReLU(alpha=.2)(x)
    x= layers.BatchNormalization()(x)
    x=layers.Flatten()(x)
    #binary_x=layers.Flatten(name='binary_flatten')(x)
    binary_output = layers.Dense(1,name='binary_output')(x)
    #multiclass_x=layers.Dense(,kernel_initializer=initializer)(x)
    #multiclass_x=layers.Flatten(name='multiclass_flatten')(multiclass_x)
    mutlticlass_output=layers.Dense(output_dim,name='multiclass_output')(x)
    #model = keras.Model(inputs=img_inputs, outputs=[binary_output,mutlticlass_output], name=name)
    model =keras.Model(inputs=img_inputs, outputs=binary_output, name=name)
    return model

^^The above functions define a discriminator

In [3]:
d=dataset_limited(['cubism'],1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.73it/s]


dataset_limited is defined in data.py; it just gets the feature maps (represented as numpy arrays) for the artistic genre

In [4]:
dick=discriminator()

In [5]:
dick.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        272       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        8224      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)      

In [6]:
def make_generator(latent_dim = 128):
    gen_1=keras.Sequential([
        keras.Input(shape=(latent_dim,)),
            # We want to generate 128 coefficients to reshape into a 7x7x128 map
            layers.Dense(2*latent_dim),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(),
            layers.Reshape((8, 8, int(2*latent_dim/64))),
            layers.Conv2DTranspose(latent_dim, (4, 4), strides=(2, 2), padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(latent_dim/2, (4, 4), strides=(2, 2), padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(latent_dim/4, (2, 2), strides=(2, 2), padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(1, (2,2), padding="same"),
            layers.BatchNormalization(name=style_blocks[0])
    ],name='gen_1')
    gen_2=keras.Sequential([
      keras.Input(shape=(latent_dim,)),
            gen_1,
            layers.Dense(64),
        layers.Conv2DTranspose(64,(2,2),strides=(2,2),padding='same'),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(),  
        layers.Conv2D(1,(2,2),padding='same'),
        layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(name=style_blocks[1]),  
    ],name='gen_2')
    gen_3=keras.Sequential([
      keras.Input(shape=(latent_dim,)),
            gen_2,
            layers.UpSampling2D(),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(name=style_blocks[2]),  
    ],name='gen_3')
    gen_4=keras.Sequential([
      keras.Input(shape=(latent_dim,)),
            gen_3,
            layers.UpSampling2D(),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(name=style_blocks[3]),  
    ],name='gen_4')
    gen_5 = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            gen_4,
            layers.Conv2D(1, (2,2), padding="same"),
            layers.LeakyReLU(alpha=0.2,name=style_blocks[4])
        ],
        name="gen_5",
    )
    return [gen_1,gen_2,gen_3,gen_4,gen_5]

A basic generator function

In [7]:
genres=random_genres(2)
labels,matrices=dataset_batched(genres,limit=50)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 24.73it/s]


random_genres is a function defined in extras.py; it just gets a bunch of random genres. dataset_batched is defined in data.py, it just gets the labels (if a painting is in 'cubism', label =cubism, represented as a one-hot encoding vector) and numpy representations of the paintings

In [8]:
matrices_zipped=tf.data.Dataset.zip(tuple(m for m in matrices))

In [9]:
class GAN(keras.Model):
    def __init__(self, discriminators, generators, latent_dim):
        super(GAN, self).__init__()
        self.discriminators = discriminators
        self.generators = generators
        self.latent_dim = latent_dim


    def compile(self, d_optimizer, g_optimizer, binary_loss_fn,multiclass_loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.binary_loss_fn = binary_loss_fn
        self.multiclass_loss_fn = multiclass_loss_fn

    def train_step(self, labels_multiclass,matrices,batch_size):
        #fake images
        layer_loss=[]
        net_d_loss=0
        net_g_loss=0
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        #generated_images = self.generator(random_latent_vectors)
        ret={}
        updated_g_vars=0
        for real_mats,block,discriminator,generator in zip(matrices,style_blocks,self.discriminators,self.generators):
            gen_mats=generator(random_latent_vectors)
            combined_mats = tf.concat([gen_mats, real_mats], axis=0)
            # Assemble labels discriminating real from fake images
            labels_binary = tf.concat(
                [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
            )
            # Add random noise to the labels - important trick!
            labels_binary += 0.05 * tf.random.uniform(tf.shape(labels_binary))
            # Train the discriminator
            with tf.GradientTape() as tape:
                #pred_binary,pred_multiclass = discriminator(combined_mats)
                pred_binary= discriminator(combined_mats)
                #pred_multiclass=pred_multiclass[-len(real_mats):]
                d_loss = self.binary_loss_fn(labels_binary, pred_binary) #+self.multiclass_loss_fn(labels_multiclass,pred_multiclass)
            grads = tape.gradient(d_loss, discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(
                zip(grads, discriminator.trainable_weights)
            )
            
            #random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            # Assemble labels that say "all real images"
            misleading_labels = tf.zeros((batch_size, 1))
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            # Train the generator
            #g_mats = tf.Variable([func(vector)[0] for vector in random_latent_vectors])
            with tf.GradientTape() as tape:
                predictions = discriminator(generator(random_latent_vectors))
                g_loss = self.binary_loss_fn( misleading_labels,predictions)
            grads = tape.gradient(g_loss, generator.trainable_weights)
            #print([sum(grad) for grad in grads])
            for g in range(updated_g_vars):
                grads[g]=tf.zeros(shape=grads[g].shape)
            updated_g_vars=len(grads)
            self.g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
            
            net_d_loss+=d_loss
            net_g_loss+=g_loss
            layer_loss.append({'g_loss':g_loss,'d_loss':d_loss})
        return layer_loss,net_g_loss,net_d_loss

defines the GAN with its own training loop

In [10]:
shapes=[(64, 64,1), (128, 128,1), (256, 256,1), (512, 512, 1), (512, 512,1)]
discriminators=[discriminator(input_shape=s,name=str(s),output_dim=len(genres)) for s in shapes]


Five discriminators, since we're using feature maps from 5 different layers

In [11]:
for d in discriminators:
    d.summary()

Model: "(64, 64, 1)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 16)        272       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 32)        8224      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 32)        

In [12]:
generators=make_generator()
for g in generators:
    g.summary()

Model: "gen_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 256)               1024      
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 4)           0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 128)       8320      
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 64)        131136

In [13]:
gan = GAN(discriminators=discriminators, generators=generators, latent_dim=128)

In [14]:
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.00003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.003),
    binary_loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
    multiclass_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
)

In [15]:
gan.generators[-1]


In [16]:
epochs=20
for epoch in range(epochs):
    print("\nStart epoch", epoch)

    step=0
    for step,(l,m) in enumerate(zip(labels,matrices_zipped)):
        # Train the discriminator & generator on one batch of real images.
        layer_loss,net_g_loss,net_d_loss= gan.train_step(l,m,10)
        print("step {}: d_loss ={} g_loss ={}".format(step,net_d_loss,net_g_loss))


Start epoch 0
step 0: d_loss =7948.01953125 g_loss =3.4659743309020996
step 1: d_loss =6285.1318359375 g_loss =3.3944640159606934
step 2: d_loss =9203.607421875 g_loss =2.836118221282959
step 3: d_loss =10525.9306640625 g_loss =1.1404080390930176
step 4: d_loss =18563.7734375 g_loss =0.20400738716125488
step 5: d_loss =7316.99267578125 g_loss =0.003850023029372096
step 6: d_loss =14144.1943359375 g_loss =0.0025097436737269163
step 7: d_loss =7895.74169921875 g_loss =0.0006103832856751978
step 8: d_loss =15418.734375 g_loss =0.013675820082426071
step 9: d_loss =14552.529296875 g_loss =0.0006455593393184245

Start epoch 1
step 0: d_loss =15257.9755859375 g_loss =0.0001518892531748861
step 1: d_loss =11341.88671875 g_loss =0.0011196843115612864
step 2: d_loss =7773.43798828125 g_loss =0.005198971834033728
step 3: d_loss =34193.75390625 g_loss =1.0333571434020996
step 4: d_loss =8868.2099609375 g_loss =12.710817337036133
step 5: d_loss =9712.7744140625 g_loss =24.917949676513672
step 6: d

step 7: d_loss =2386.109375 g_loss =8.631892204284668
step 8: d_loss =21823.99609375 g_loss =9.077024459838867
step 9: d_loss =11137.470703125 g_loss =4.801805019378662

Start epoch 14
step 0: d_loss =3636.5126953125 g_loss =5.887483596801758
step 1: d_loss =5331.193359375 g_loss =1.1192866563796997
step 2: d_loss =6407.94140625 g_loss =5.529237270355225
step 3: d_loss =8887.9931640625 g_loss =2.270313262939453
step 4: d_loss =16951.572265625 g_loss =2.005033016204834
step 5: d_loss =6980.1318359375 g_loss =1.5734823942184448
step 6: d_loss =15432.7998046875 g_loss =4.27389669418335
step 7: d_loss =8357.404296875 g_loss =1.2536100149154663
step 8: d_loss =9332.6484375 g_loss =4.7569427490234375
step 9: d_loss =15348.3955078125 g_loss =5.7549943923950195

Start epoch 15
step 0: d_loss =11237.6357421875 g_loss =6.0348968505859375
step 1: d_loss =20589.08984375 g_loss =8.094359397888184
step 2: d_loss =9133.220703125 g_loss =7.301883220672607
step 3: d_loss =16347.6123046875 g_loss =8.543

In [17]:
def test_optimizers(d_optimizer,g_optimizer,epochs=10,GAN=GAN):
    discriminators=[discriminator(input_shape=s,name=str(s),output_dim=len(genres)) for s in shapes]
    generators=make_generator()
    gan = GAN(discriminators=discriminators, generators=generators, latent_dim=128)
    gan.compile(
        d_optimizer=d_optimizer,
        g_optimizer=g_optimizer,
        binary_loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
        multiclass_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    )
    for epoch in range(epochs):
        print("\nStart epoch", epoch)

        step=0
        for step,(l,m) in enumerate(zip(labels,matrices_zipped)):
            # Train the discriminator & generator on one batch of real images.
            layer_loss,net_g_loss,net_d_loss= gan.train_step(l,m,10)
            print("step {}: d_loss ={} g_loss ={}".format(step,net_d_loss,net_g_loss))

In [20]:
test_optimizers(
    keras.optimizers.Adam(learning_rate=0.003,clipnorm=1),keras.optimizers.Adam(learning_rate=0.003,clipnorm=.1),epochs=15)


Start epoch 0
step 0: d_loss =6293.24755859375 g_loss =3.4095864295959473
step 1: d_loss =12484892.0 g_loss =3.551870822906494
step 2: d_loss =56538.015625 g_loss =3.2099716663360596
step 3: d_loss =118615.2734375 g_loss =14.848846435546875
step 4: d_loss =370862.15625 g_loss =69.11424255371094
step 5: d_loss =4257733.5 g_loss =12.385921478271484
step 6: d_loss =388834.875 g_loss =118.61106872558594
step 7: d_loss =590780.375 g_loss =6.285704612731934
step 8: d_loss =674149.5625 g_loss =91.84060668945312
step 9: d_loss =858037.25 g_loss =97.4207534790039

Start epoch 1
step 0: d_loss =1073855.0 g_loss =66.67455291748047
step 1: d_loss =308587.375 g_loss =56.098487854003906
step 2: d_loss =1063334.25 g_loss =38.314605712890625
step 3: d_loss =2583110.0 g_loss =29.885940551757812
step 4: d_loss =2283411.0 g_loss =42.5042724609375
step 5: d_loss =858156.8125 g_loss =120.31988525390625
step 6: d_loss =197185.0625 g_loss =281.8813171386719
step 7: d_loss =12115778.0 g_loss =64.138977050781

In [21]:
test_optimizers(
    keras.optimizers.Adam(learning_rate=0.03,clipnorm=1),keras.optimizers.Adam(learning_rate=0.03,clipnorm=1),epochs=15
)


Start epoch 0
step 0: d_loss =4613.5478515625 g_loss =767.9803466796875
step 1: d_loss =207150720.0 g_loss =4805.52001953125
step 2: d_loss =32683208.0 g_loss =4435.7607421875
step 3: d_loss =1786087936.0 g_loss =3773.489013671875
step 4: d_loss =374036640.0 g_loss =2550.514404296875
step 5: d_loss =84495792.0 g_loss =5352.91650390625
step 6: d_loss =24789739520.0 g_loss =14522.79296875
step 7: d_loss =743788800.0 g_loss =126288.2421875
step 8: d_loss =698498560.0 g_loss =775298.5
step 9: d_loss =19330107392.0 g_loss =286383.9375

Start epoch 1
step 0: d_loss =4619820032.0 g_loss =219692.71875
step 1: d_loss =9391558656.0 g_loss =42175.734375
step 2: d_loss =19702935552.0 g_loss =204895.21875
step 3: d_loss =36318609408.0 g_loss =173845.765625
step 4: d_loss =12892940288.0 g_loss =311537.25
step 5: d_loss =3764527616.0 g_loss =813815.125
step 6: d_loss =3225978624.0 g_loss =1645239.875
step 7: d_loss =4963955712.0 g_loss =1006329.8125
step 8: d_loss =405736768.0 g_loss =883191.4375
st

In [22]:
test_optimizers(
    keras.optimizers.Adam(learning_rate=0.0003,clipnorm=1),keras.optimizers.Adam(learning_rate=0.0003,clipnorm=1,epochs=15)
)

TypeError: Unexpected keyword argument passed to optimizer: epochs

In [ ]:
class GANoneGen(keras.Model):
    def __init__(self, discriminators, generators, latent_dim):
        super(GAN, self).__init__()
        self.discriminators = discriminators
        self.generators = generators
        self.latent_dim = latent_dim


    def compile(self, d_optimizer, g_optimizer, binary_loss_fn,multiclass_loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.binary_loss_fn = binary_loss_fn
        self.multiclass_loss_fn = multiclass_loss_fn

    def train_step(self, labels_multiclass,matrices,batch_size):
        #fake images
        layer_loss=[]
        net_d_loss=0
        net_g_loss=0
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        #generated_images = self.generator(random_latent_vectors)
        ret={}
        updated_g_vars=0
        for real_mats,block,discriminatorin zip(matrices,style_blocks,self.discriminators):
            gen_mats=generator(random_latent_vectors)
            combined_mats = tf.concat([gen_mats, real_mats], axis=0)
            # Assemble labels discriminating real from fake images
            labels_binary = tf.concat(
                [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
            )
            # Add random noise to the labels - important trick!
            labels_binary += 0.05 * tf.random.uniform(tf.shape(labels_binary))
            # Train the discriminator
            with tf.GradientTape() as tape:
                #pred_binary,pred_multiclass = discriminator(combined_mats)
                pred_binary= discriminator(combined_mats)
                #pred_multiclass=pred_multiclass[-len(real_mats):]
                d_loss = self.binary_loss_fn(labels_binary, pred_binary) #+self.multiclass_loss_fn(labels_multiclass,pred_multiclass)
            grads = tape.gradient(d_loss, discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(
                zip(grads, discriminator.trainable_weights)
            )
            
            #random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            # Assemble labels that say "all real images"
            misleading_labels = tf.zeros((batch_size, 1))
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            # Train the generator
            #g_mats = tf.Variable([func(vector)[0] for vector in random_latent_vectors])
            with tf.GradientTape() as tape:
                predictions = discriminator(generator(random_latent_vectors))
                g_loss = self.binary_loss_fn( misleading_labels,predictions)
            grads = tape.gradient(g_loss, generator.trainable_weights)
            #print([sum(grad) for grad in grads])
            for g in range(updated_g_vars):
                grads[g]=tf.zeros(shape=grads[g].shape)
            updated_g_vars=len(grads)
            self.g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
            
            net_d_loss+=d_loss
            net_g_loss+=g_loss
            layer_loss.append({'g_loss':g_loss,'d_loss':d_loss})
        return layer_loss,net_g_loss,net_d_loss